Демонстрационные примеры для задач линейной регрессии

In [11]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RANSACRegressor
from ipywidgets import interact
from ipywidgets.widgets import FloatSlider, IntSlider



In [12]:
@interact(a=FloatSlider(min=-2, max=2, step=0.1, value=1),
          b=FloatSlider(min=-1, max=1, step=0.1, value=0),
          e=FloatSlider(min=0, max=2, step=0.01, value=0.1),
          n=IntSlider(min=2, max=200, step=2, value=50))
def linear_regression_1d(a, b, e=0.5, n=100):
    """
    Оцениваем по формуле параметры a и b одномерной линейной регрессии: y = a*x + b
    e : уровень шума
    n : количество точек
    """
    xmin = 0
    xmax = 5
    np.random.seed(42)
    x = np.random.rand(n)*(xmax - xmin) + xmin
    y = a*x + b + e*np.random.randn(n)
    a_hat = ((x - x.mean())*(y - y.mean())).sum()/((x - x.mean())**2).sum()
    # a_hat = (x*(y - y.mean())).sum()/(x*(x - x.mean())).sum()    
    b_hat = y.mean() - a_hat*x.mean()
    y_pred = a_hat*x + b_hat
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    plt.scatter(x, y)
    plt.plot([x.min(), x.max()], [x.min()*a + b, x.max()*a + b], label='true', c='b')
    plt.plot([x.min(), x.max()], [x.min()*a_hat + b_hat, x.max()*a_hat + b_hat], label='pred', c='r')
    plt.ylim(-12, 12)
    plt.legend()
    plt.title(f'a\u0302={a_hat:.5} b\u0302={b_hat:.5} r2={r2:.5} mse={mse:.5} mae={mae:.5}')
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='a', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…

In [13]:
from mpl_toolkits.mplot3d import Axes3D

@interact(a=FloatSlider(min=-2, max=2, step=0.1, value=1),
          b=FloatSlider(min=-2, max=2, step=0.1, value=1),
          c=FloatSlider(min=-1, max=1, step=0.1, value=0),
          e=FloatSlider(min=0, max=2, step=0.01, value=0.1),
          n=IntSlider(min=3, max=200, step=3, value=50))
def linear_regression_2d(a, b, c, e=0.5, n=100):
    """
    Оцениваем параметры a, b и с двумерной линейной регрессии: y = a*x1 + b*x2 + c
    e : уровень шума
    n : количество точек
    """
    xmin = 0
    xmax = 5
    np.random.seed(42)
    x0 = np.ones(n)
    x1 = np.random.rand(n)*(xmax - xmin) + xmin
    x2 = np.random.rand(n)*(xmax - xmin) + xmin
    y = a*x1 + b*x2 + c + e*np.random.randn(n)
    xt = np.stack((x0, x1, x2))
    x = xt.transpose()
    w_hat = np.linalg.inv(xt@x)@xt@y
    c_hat, a_hat, b_hat = w_hat.tolist()
    print(w_hat)
    y_pred = a_hat*x1 + b_hat*x2 + c_hat
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x1, x2, y)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_zlabel('y')
    plt.ylim(-12, 12)
    plt.title(f'a\u0302={a_hat:.4} b\u0302={b_hat:.4} c\u0302={c_hat:.4} ' + 
              f'r2={r2:.4} mse={mse:.4} mae={mae:.4}')
    plt.show()
    

interactive(children=(FloatSlider(value=1.0, description='a', max=2.0, min=-2.0), FloatSlider(value=1.0, descr…

In [10]:
@interact(a=FloatSlider(min=-2, max=2, step=0.1, value=1),
          b=FloatSlider(min=-2, max=2, step=0.1, value=1),
          c=FloatSlider(min=-1, max=1, step=0.1, value=0),
          k=FloatSlider(min=0, max=0.5, step=0.01, value=0.5),
          e=FloatSlider(min=0, max=2, step=0.01, value=0.1),
          n=IntSlider(min=3, max=200, step=3, value=50))
def linear_regression_2d_multicollinearity(a, b, c, k, e=0.5, n=100):
    """
    Оцениваем параметры a, b и с двумерной линейной регрессии: y = a*x1 + b*x2 + c
    Моделирование полной или частичной мультиколлинеарности (k=0 для полной).
    Используем различные способы оценки.
    e : уровень шума
    n : количество точек
    """
    xmin = 0
    xmax = 5
    np.random.seed(42)
    x0 = np.ones(n)
    x1 = np.random.rand(n)*(xmax - xmin) + xmin
    x2 = x1 + k*np.random.randn(n)
    y = a*x1 + b*x2 + c + e*np.random.randn(n)
    
    # вычислим коэффициенты регрессии по формуле (XT@X)^-1@XT@Y
    xt = np.stack((x0, x1, x2))
    x = xt.transpose()
    w_hat = np.zeros(3)
    try:
        w_hat = np.linalg.inv(xt@x)@xt@y
    except BaseException as ex:
        print(f'Raw formula error: {ex}')
    c_hat, a_hat, b_hat = w_hat.tolist()
    print(f'w_hat={w_hat} Raw formula')
    
    # вычислим псевдообратную матрицу с помощью SVD:
    u, s, vh = np.linalg.svd(x)
    s_inv = np.identity(3)*(s**-1)
    x_pi = vh.transpose()@s_inv@u.transpose()[:3,:]
    
    # вычислим коэффициенты регрессии с помощью псевдообратной матрицы:
    w_hat_pi = x_pi@y
    print(f'w_hat={w_hat_pi} Pseudoinverse matrix')
    
    # вычислим коэффиценты регрессии с помощью LinearRegression
    linear_regression = LinearRegression()
    linear_regression.fit(x, y)
    w_hat_lr = np.hstack((np.array(linear_regression.intercept_), linear_regression.coef_[1:]))
    print(f'w_hat={w_hat_lr} LinearRegression')
    
    # вычислим коэффиценты регрессии с помощью Ridge
    ridge = Ridge()
    ridge.fit(x, y)
    w_hat_ridge = np.hstack((np.array(ridge.intercept_), ridge.coef_[1:]))
    print(f'w_hat={w_hat_ridge} Ridge')
    
    # вычислим коэффиценты регрессии с помощью LASSO
    lasso = Lasso()
    lasso.fit(x, y)
    w_hat_lasso = np.hstack((np.array(lasso.intercept_), lasso.coef_[1:]))
    print(f'w_hat={w_hat_lasso} LASSO')
    
    # выведем сингулярные числа ():
    print(f'\u03bb**0.5={s}')
    # выведем собственные значения матрицы XT@X:
    print(f'\u03bb(XT@X)={s**2}  {round((s**2).max()/(s**2).min(),3)}')
    
    y_pred = a_hat*x1 + b_hat*x2 + c_hat
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    plt.scatter(x1, x2)
    plt.xlabel('x1')
    plt.xlim(-1, 6)
    plt.ylabel('x2')
    plt.ylim(-1, 6)
    plt.title(f'a\u0302={a_hat:.5} b\u0302={b_hat:.5} c\u0302={c_hat:.4} ' + 
              f'r2={r2:.5} mse={mse:.5} mae={mae:.5}')
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='a', max=2.0, min=-2.0), FloatSlider(value=1.0, descr…

In [14]:
@interact(a=FloatSlider(min=-2, max=2, step=0.1, value=-2),
          b=FloatSlider(min=-1, max=1, step=0.1, value=-1),
          e=FloatSlider(min=0, max=2, step=0.01, value=0.1),
          n=IntSlider(min=2, max=200, step=2, value=30),
          ox=FloatSlider(min=0, max=5, step=0.1, value=2.5),
          oy=FloatSlider(min=-2, max=2, step=0.1, value=0.5),
          on=IntSlider(min=0, max=20, step=1, value=5))
def linear_regression_1d_ransac(a, b, e=0.5, n=100, ox=0.5, oy=0.5, on=5):
    """
    Оцениваем с помощью RANSAC параметры a и b одномерной линейной регрессии: y = a*x + b
    e : уровень шума
    n : количество точек
    ox, oy : координаты центра выборосов
    on : количество точек с выбросом
    """
    xmin = 0
    xmax = 5
    np.random.seed(42)
    x = np.random.rand(n)*(xmax - xmin) + xmin
    y = a*x + b + e*np.random.randn(n)
    xo = np.random.randn(on) + ox
    yo = np.random.randn(on) + oy
    X = np.hstack((x, xo)).reshape((n + on, 1))
    Y = np.hstack((y, yo))
    ransac_regressor = RANSACRegressor()
    ransac_regressor.fit(X,Y)
    a_hat = ransac_regressor.estimator_.coef_[0]
    b_hat = ransac_regressor.estimator_.intercept_
    
    linear_regression = LinearRegression()
    linear_regression.fit(X,Y)
    lr_a_hat = linear_regression.coef_[0]
    lr_b_hat = linear_regression.intercept_
    
    y_pred = a_hat*x + b_hat
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    plt.scatter(x, y, c='b', marker='o')
    plt.scatter(xo, yo, c='r', marker='^')
    plt.plot([x.min(), x.max()], [x.min()*a + b, x.max()*a + b], label='true', c='b')
    plt.plot([x.min(), x.max()], [x.min()*a_hat + b_hat, x.max()*a_hat + b_hat], label='RANSAC', c='r')
    plt.plot([x.min(), x.max()], [x.min()*lr_a_hat + lr_b_hat, x.max()*lr_a_hat + lr_b_hat], label='Linear regression', c='g')
    plt.ylim(-12, 12)
    plt.legend()
    plt.title(f'a\u0302={a_hat:.5} b\u0302={b_hat:.5} r2={r2:.5} mse={mse:.5} mae={mae:.5}')
    plt.show()


interactive(children=(FloatSlider(value=-2.0, description='a', max=2.0, min=-2.0), FloatSlider(value=-1.0, des…

## Huber Loss 

In [19]:
from scipy.special import huber
from sklearn.linear_model import HuberRegressor

In [25]:
@interact(a=FloatSlider(min=-2, max=2, step=0.1, value=-2),
          b=FloatSlider(min=-1, max=1, step=0.1, value=-1),
          e=FloatSlider(min=0, max=2, step=0.01, value=0.1),
          n=IntSlider(min=2, max=200, step=2, value=30),
          ox=FloatSlider(min=0, max=5, step=0.1, value=2.5),
          oy=FloatSlider(min=-2, max=2, step=0.1, value=0.5),
          on=IntSlider(min=0, max=20, step=1, value=5))
def linear_regression_1d_ransac(a, b, e=0.5, n=100, ox=0.5, oy=0.5, on=5):
    """
    Оцениваем с помощью RANSAC параметры a и b одномерной линейной регрессии: y = a*x + b
    e : уровень шума
    n : количество точек
    ox, oy : координаты центра выборосов
    on : количество точек с выбросом
    """
    xmin = 0
    xmax = 5
    np.random.seed(42)
    x = np.random.rand(n)*(xmax - xmin) + xmin
    y = a*x + b + e*np.random.randn(n)
    xo = np.random.randn(on) + ox
    yo = np.random.randn(on) + oy
    X = np.hstack((x, xo)).reshape((n + on, 1))
    Y = np.hstack((y, yo))
    ransac_regressor = RANSACRegressor()
    ransac_regressor.fit(X,Y)
    a_hat = ransac_regressor.estimator_.coef_[0]
    b_hat = ransac_regressor.estimator_.intercept_
    
    linear_regression = LinearRegression()
    linear_regression.fit(X,Y)
    lr_a_hat = linear_regression.coef_[0]
    lr_b_hat = linear_regression.intercept_
    
    #
    huber = HuberRegressor().fit(X,Y)
    lr_a_hu = huber.coef_[0]
    lr_b_hu = huber.intercept_
    #
    
    y_pred = a_hat*x + b_hat
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    #hu = huber (y, y_pred)
    
    #print (hu)
    
    plt.scatter(x, y, c='b', marker='o')
    plt.scatter(xo, yo, c='r', marker='^')
    plt.plot([x.min(), x.max()], [x.min()*a + b, x.max()*a + b], label='true', c='b')
    plt.plot([x.min(), x.max()], [x.min()*a_hat + b_hat, x.max()*a_hat + b_hat], label='RANSAC', c='r')
    plt.plot([x.min(), x.max()], [x.min()*lr_a_hat + lr_b_hat, x.max()*lr_a_hat + lr_b_hat], label='Linear regression', c='g')
    
    plt.plot([x.min(), x.max()], [x.min()*lr_a_hu + lr_b_hu, x.max()*lr_a_hu + lr_b_hu], label='Huber regression', c='c')
    
    plt.ylim(-12, 12)
    plt.legend()
    #plt.title(f'a\u0302={a_hat:.5} b\u0302={b_hat:.5} r2={r2:.5} mse={mse:.5} mae={mae:.5} hu={hu:.5}' )
    plt.title(f'a\u0302={a_hat:.5} b\u0302={b_hat:.5} r2={r2:.5} mse={mse:.5} mae={mae:.5}' )
    plt.show()


interactive(children=(FloatSlider(value=-2.0, description='a', max=2.0, min=-2.0), FloatSlider(value=-1.0, des…